In [1]:
import pandas as pd
import csv
import pycountry
# Load the CSV file into a DataFrame
tax_data_df = pd.read_csv('F:\YinYang\BIA utbildning\Kurs 5 Design och Visualisering\Vecka 4\orgAvg personal income tax and social security contributions on gross labour income.csv')

In [2]:
# Filter the dataset to keep only the latest row per country based on the 'Year' column
tax_data_df = tax_data_df.sort_values(by=['Country', 'Year'], ascending=[True, False]).drop_duplicates(subset=['Country'], keep='first')

Intressant att CGPT valde att göra allt på en rad för första gången. Det är inte lika läsbart som när den delar upp det.

Lite kåseri om koden:
Sorteringen ascending=[True, False] innebär att Country är ascending. A är först i alfabetet 1,2,3 = a,b,c
Year är inte ascending, i den här implementeringen blir ett false på ascending att det är det descending. 
Rent tekniskt utvärderar alla sorteringar som inte strikt fyller kravet ascending = False.
Det är kul att implementeringen sorterar descending om ascending ska vara False, den kunde ha lämnat det osorterat? 
(men då behöver den kolla att det inte råkar bli ascending, så det skulle inte spara allt arbete den nu gör, oavsett.)

In [3]:
#Massa kolumner, låt oss rensa
tax_data_df.head()

,COU,Country,INCOMEAW,Income as a percentage of the average wage,AVGRATES,Average tax rates and wedges,YEA,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
22,AUS,Australia,67,67,TOT_TAX_WEDGE,Total tax wedge,2022,2022,PC,Percentage,0,Units,NaN,NaN,21.234256,NaN,NaN
114,AUT,Austria,67,67,TOT_TAX_WEDGE,Total tax wedge,2022,2022,PC,Percentage,0,Units,NaN,NaN,43.360751,NaN,NaN
206,BEL,Belgium,67,67,TOT_TAX_WEDGE,Total tax wedge,2022,2022,PC,Percentage,0,Units,NaN,NaN,46.500027,NaN,NaN
298,CAN,Canada,67,67,TOT_TAX_WEDGE,Total tax wedge,2022,2022,PC,Percentage,0,Units,NaN,NaN,29.953667,NaN,NaN
2782,CHL,Chile,67,67,TOT_TAX_WEDGE,Total tax wedge,2022,2022,PC,Percentage,0,Units,NaN,NaN,7.000000,NaN,NaN


In [4]:
#En ny DataFrame som bara innehåller kolumnerna som angivits i EnDataFrame[['kolumn', 'kolunm']] 
reduced_tax_data_df = tax_data_df[['Country', 'Value']].copy()

# Show the first few rows of the filtered data
reduced_tax_data_df.head()

,Country,Value
22,Australia,21.234256
114,Austria,43.360751
206,Belgium,46.500027
298,Canada,29.953667
2782,Chile,7.000000


Jag tycker "double square brackets" är en intressant pandas grej.
Tänkte att det betyder kanske [[skit i allt utom oss]]. I kontext ovan funkar det att tänka så, eftersom jag ska spara vissa kolumner. Men det är trubbigt tänkt. [[Jag vill ha en DataFrame]] är mer korrekt tänkt.

.copy är för att det annars är en slice av DF:en den kom ifrån, vilket den inte gillar att fortsätta arbeta med utan att varna. Antar att slice:s är tacksamma att arbeta med när det är stora dataset. antar att pandas minns förändringarna då, utan att skapa fullständiga kopior, vilket behövs för att inte behöva vänta på stora kopior, och i värsta fall slå i kapacitetgränser på lagringsmedia när det är mycket data som hanteras. I cloud sammanhang där man betalar för lagring kan det bli dyrt att kopiera stora dataset till höger och vänster utan att tänka sig för.

In [5]:
#Dags att addera två bokstavs ISO kod. Återanvänder koden för COLI

#Koden läser csv fil, så filen över behöver sparas som sådan först.
intermediate_tax_df_file_path = 'F:\YinYang\BIA utbildning\Kurs 5 Design och Visualisering\Vecka 7\TOS_wout_CountryISO.csv'
reduced_tax_data_df.to_csv(intermediate_tax_df_file_path, index=False)

special_cases = {
    'Bolivia': 'BO',
    'Iran': 'IR',
    'Ivory Coast': 'CI',
    'Kosovo (Disputed Territory)': 'XK',  # Not an official ISO code
    'Moldova': 'MD',
    'Palestine': 'PS',
    'Russia': 'RU',
    'South Korea': 'KR',
    'Syria': 'SY',
    'Taiwan': 'TW',
    'Tanzania': 'TZ',
    'Venezuela': 'VE',
    'Vietnam': 'VN',
    'Czech Republic': 'CZ'
}


def get_two_letter_code(country):
    # Try to get the country code from pycountry
    try:
        return pycountry.countries.get(name=country).alpha_2
    except AttributeError:
        # Check the special cases dictionary
        return special_cases.get(country, "Unknown")


# Read the existing CSV file
with open('F:\YinYang\BIA utbildning\Kurs 5 Design och Visualisering\Vecka 7\TOS_wout_CountryISO.csv', 'r') as infile:

    reader = csv.reader(infile)
    header = next(reader)  # Read the header
    
    # Add the 'Country Code' to the header
    header.append('Country Code')

    # Create a new CSV file to write the updated data
    with open('f:/YinYang/BIA utbildning/Kurs 5 Design och Visualisering/Vecka 7/TOS.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        
        # Write the new header
        writer.writerow(header)
        
        # Iterate over each row in the old file
        for row in reader:
            country = row[0]  # Assuming 'Country' is the second column
            country_code = get_two_letter_code(country)
            
            # Add the country code to this row
            row.append(country_code)
            
            # Write the updated row to the new file
            writer.writerow(row)


In [6]:
#Sista steget, spara
#new_tax_df_file_path = 'F:\YinYang\BIA utbildning\Kurs 5 Design och Visualisering\Vecka 8\TOS.csv'
#new_tax_df.to_csv(new_tax_df_file_path, index=False)